Ce notebook transforme et adapte le tutoriel « Déployer Apache Airflow sur AKS avec Helm » pour exécution étape par étape.

Objectif
Déployer Apache Airflow sur Azure Kubernetes Service (AKS) à l'aide de Helm, puis accéder à l'interface web d'Airflow.

Avertissement
Les cellules avec des commandes az, kubectl et helm doivent être exécutées dans un environnement qui dispose déjà de ces outils et d'autorisations Azure suffisantes. Dans un environnement Jupyter standard, vous pouvez souvent les exécuter via %%bash ou en préfixant par ! si les binaires sont installés.


Prérequis
1. Abonnement Azure actif avec permissions suffisantes (contributeur) sur le groupe de ressources ciblé.
2. Azure CLI 2.61.0 ou version ultérieure.
3. Helm 3 ou version ultérieure.
4. kubectl.
5. Un registre Git ou un stockage des DAG selon vos besoins.
6. Optionnel mais recommandé : Microsoft Entra Workload ID pour l'accès aux ressources Azure depuis les Pods.


In [ ]:
%%bash
set -euo pipefail
echo "Versions des outils"
az version | head -n 5 || echo "Azure CLI non disponible ici"
helm version || echo "Helm non disponible ici"
kubectl version --client || echo "kubectl non disponible ici"


In [ ]:
%%bash
# Connexion à Azure (si nécessaire). Exécutez dans un terminal interactif.
# az login
# Si vous avez plusieurs abonnements, sélectionnez celui à utiliser :
# az account set --subscription "<ID ou Nom de l'abonnement>"


In [ ]:
# Paramètres personnalisables pour votre déploiement
RESOURCE_GROUP = "rg-airflow-aks"
LOCATION = "westeurope"
CLUSTER_NAME = "aks-airflow-demo"
K8S_VERSION = ""  # laissez vide pour version par défaut
AIRFLOW_NAMESPACE = "airflow"
RELEASE_NAME = "airflow"
INGRESS_HOST = "airflow.example.com"  # à adapter si vous activez un ingress


In [ ]:
%%bash
set -euo pipefail
echo "Création du groupe de ressources"
az group create -n "${RESOURCE_GROUP}" -l "${LOCATION}"


In [ ]:
%%bash
set -euo pipefail
echo "Création du cluster AKS (nœud système x1 Standard_D4s_v5, pool user x2 Standard_D4s_v5)"
# Activez Workload Identity et OIDC pour de bonnes pratiques modernes d'authentification
az aks create   --resource-group "${RESOURCE_GROUP}"   --name "${CLUSTER_NAME}"   --location "${LOCATION}"   --node-count 1   --node-vm-size Standard_D4s_v5   --enable-oidc-issuer   --enable-workload-identity   ${K8S_VERSION:+--kubernetes-version ${K8S_VERSION}}   --generate-ssh-keys

# Exemple de pool utilisateur additionnel (optionnel)
az aks nodepool add   --resource-group "${RESOURCE_GROUP}"   --cluster-name "${CLUSTER_NAME}"   --name userpool   --node-count 2   --node-vm-size Standard_D4s_v5


In [ ]:
%%bash
set -euo pipefail
echo "Récupération des identifiants du cluster"
az aks get-credentials -g "${RESOURCE_GROUP}" -n "${CLUSTER_NAME}" --overwrite-existing
kubectl get nodes -o wide


In [ ]:
%%bash
set -euo pipefail
echo "Création du namespace Airflow"
kubectl create namespace "${AIRFLOW_NAMESPACE}" || true


In [ ]:
%%bash
set -euo pipefail
echo "Ajout du dépôt Helm Apache Airflow et mise à jour"
helm repo add apache-airflow https://airflow.apache.org
helm repo update
helm search repo apache-airflow


In [ ]:
%%bash
cat > values-airflow.yaml <<'YAML'
# Valeurs minimales d'exemple pour déployer Airflow sur AKS.
# Ajustez selon vos besoins de production.

executor: "KubernetesExecutor"

# Active la base Postgres fournie par la chart pour démarrer rapidement.
# Pour la prod, préférez un service managé (Azure Database for PostgreSQL).
postgresql:
  enabled: true

# Persistance des logs via PVC (Azure Files/Disks selon votre StorageClass par défaut).
logs:
  persistence:
    enabled: true
    existingClaim: ""
    size: 10Gi

# Création d'un compte admin par défaut (changez les identifiants après le déploiement).
webserver:
  defaultUser:
    enabled: true
    username: admin
    password: admin
  service:
    type: ClusterIP

# Exemple d'ingress (désactivé par défaut). À activer si vous avez un contrôleur Ingress.
ingress:
  enabled: false
  web:
    hosts:
      - ${INGRESS_HOST}
    tls: []
YAML


In [ ]:
%%bash
set -euo pipefail
echo "Déploiement d'Airflow via Helm"
helm upgrade --install "${RELEASE_NAME}" apache-airflow/airflow   --namespace "${AIRFLOW_NAMESPACE}"   -f values-airflow.yaml

echo "Attente des Pods"
kubectl wait --namespace "${AIRFLOW_NAMESPACE}"   --for=condition=Ready pods --all --timeout=10m || true

kubectl get pods -n "${AIRFLOW_NAMESPACE}"


In [ ]:
%%bash
set -euo pipefail
echo "Accès à l'interface web d'Airflow via port-forward"
echo "Exécutez : kubectl port-forward svc/airflow-webserver 8080:8080 -n ${AIRFLOW_NAMESPACE}"
echo "Puis ouvrez http://localhost:8080 (identifiants: admin / admin)"


In [ ]:
%%bash
cat > example_dag.py <<'PY'
from datetime import datetime
from airflow import DAG
from airflow.operators.bash import BashOperator

with DAG(
    dag_id="bonjour_aks",
    start_date=datetime(2024, 1, 1),
    schedule=None,
    catchup=False,
    tags=["demo"],
):
    t = BashOperator(task_id="echo", bash_command="echo 'Bonjour AKS + Airflow'")
PY

echo "Copiez ce fichier dans le dossier de DAGs du déploiement."
echo "Selon votre configuration Helm, montez un PVC pour /opt/airflow/dags ou utilisez un dépôt Git/Sync."


In [ ]:
%%bash
set -euo pipefail
echo "Nettoyage des ressources (optionnel)"
echo "Suppression de la release Helm"
helm uninstall "${RELEASE_NAME}" -n "${AIRFLOW_NAMESPACE}" || true
echo "Suppression du groupe de ressources complet (irréversible)"
# az group delete -n "${RESOURCE_GROUP}" --yes --no-wait


Bonnes pratiques et remarques
1. Sécurité et identités : privilégiez Microsoft Entra Workload ID pour accéder à Key Vault, Storage, etc., depuis vos Pods sans secrets statiques.
2. Base de données : pour la production, utilisez Azure Database for PostgreSQL Flexible Server et désactivez le sous-chart postgresql.
3. Stockage des logs : pour des logs persistants et partageables, utilisez un PVC basé sur Azure Files ou un export vers Azure Blob via sidecar/remote logging.
4. Observabilité : activez Prometheus/Grafana ou Azure Monitor Container Insights pour suivre l'état des composants Airflow et du cluster.
5. Disponibilité : répartissez les nœuds sur plusieurs zones, configurez des ressources requests/limits pour les composants Airflow, et envisagez l'autoscaling.


Références
1. Déployer Apache Airflow sur AKS avec Helm (Microsoft Learn).
2. Documentation Airflow Helm Chart.
3. Workload Identity sur AKS (Microsoft Learn).
